In [1]:
#!/usr/bin/env python
################################################################
#  Copyright (C) 2015 OpenEye Scientific Software, Inc.
################################################################
from __future__ import print_function
import os, sys
import numpy as np

In [2]:
import openeye.oechem as oechem

In [3]:
inFileName = 'WNE6S-warmup.oeb.gz'
ifs = oechem.oemolistream(inFileName)
mol = oechem.OEMol()
oechem.OEReadMolecule(ifs, mol)

True

In [4]:
print( mol.NumAtoms())

32357


In [5]:
outFileName = 'out.pdb'
ofs = oechem.oemolostream(outFileName)
flavor = ofs.GetFlavor(oechem.OEFormat_PDB) ^ oechem.OEOFlavor_PDB_OrderAtoms
ofs.SetFlavor(oechem.OEFormat_PDB, flavor)
oechem.OEWriteConstMolecule(ofs, mol)
ofs.close()

In [6]:
from simtk import unit, openmm
from simtk.openmm import app
from simtk.openmm import XmlSerializer

import parmed
import base64
import pickle

import json
import plmd
import LoggingStopwatch

In [7]:
structTag = 'structure'
if structTag in mol.GetData().keys():
    print('mol GetData finds tag:', structTag)
    encoded_struct = mol.GetData( structTag)

mol GetData finds tag: structure


In [8]:
decoded_structure = base64.b64decode( encoded_struct)
struct_dict = pickle.loads(decoded_structure)
struct = parmed.structure.Structure()
struct.__setstate__(struct_dict)
#help( struct)

In [9]:
topology = struct.topology
initPositions = struct.positions
#help( topology)

In [10]:
#help( initPositions)
print(initPositions[0])

(33.43, 11.279999999999999, 44.490000000000009) A


In [11]:
#return openmm.XmlSerializer.deserialize(serialized_system)
systemTag = 'system'
if systemTag in mol.GetData().keys():
    print('mol GetData finds tag:', systemTag)
    serialized_system = mol.GetData( systemTag)

mol GetData finds tag: system


In [12]:
system = openmm.XmlSerializer.deserialize(serialized_system)
#help( system)

In [13]:
if 'state' in mol.GetData().keys():
    print('mol GetData finds tag:', 'state')
    mol.GetData(oechem.OEGetTag('state'))
    serialized_state = mol.GetData(oechem.OEGetTag('state'))

mol GetData finds tag: state


In [14]:
state = openmm.XmlSerializer.deserialize( serialized_state )

In [17]:
pos = state.getPositions()
print( len( pos))
print( pos[0])
veloc = state.getVelocities()

32357
(3.3569929599761963, 1.1250951290130615, 4.444279670715332) nm


TypeError: Velocities were not requested in getState() call, so are not available.

In [16]:
xvec, yvec, zvec = state.getPeriodicBoxVectors()
print( xvec)
print( yvec)
print( zvec)

(6.995734810829163, 0.0, 0.0) nm
(0.0, 6.995734810829163, 0.0) nm
(0.0, 0.0, 6.995734810829163) nm
